# THE POPE DATASET CREATION

In [1]:
import pandas as pd

### Scraping Twitter 

In [2]:
import snscrape.modules.twitter as sntwitter
import csv
maxTweets = 20000

csvFile = open('Pope.csv', 'a', newline='', encoding='utf8')

csvWriter = csv.writer(csvFile)
csvWriter.writerow(['id','date','tweet',])

for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:@Pontifex until:2020-11-01-filter:replies').get_items()):
    if i > maxTweets :
        break
    csvWriter.writerow([tweet.id, tweet.date, tweet.content])

csvFile.close()

In [3]:
pope = pd.read_csv('Pope.csv')
pope.shape

(2877, 3)

In [4]:
pope = pope.drop_duplicates(subset = 'id')

In [5]:
pope.shape

(2877, 3)

In [7]:
pope.date = pd.to_datetime(pope.date)

In [8]:
pope.groupby([pope['date'].dt.year])['id'].count()

date
2013    230
2014    248
2015    220
2016    362
2017    352
2018    397
2019    468
2020    600
Name: id, dtype: int64

### Likes and Retweets

In [9]:
links_to_scrape = pope['id'].tolist()

In [10]:
total_count = len(links_to_scrape)
chunks = (total_count - 1) // 50 + 1

In [11]:
import tweepy

consumer_key = 
consumer_secret = 
access_token =  
access_token_secret = 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [13]:
def fetch_tw(links_to_scrape):
    
    list_of_tw_status = api.statuses_lookup(links_to_scrape, tweet_mode= "extended")
    
    empty_data = pd.DataFrame()
    
    for status in list_of_tw_status:
            tweet_elem = {"tweet_id": str(status.id),
                          "tweet":status.full_text,
                          'likes': status.favorite_count, 
                          'retweets': status.retweet_count,
                          "date":status.created_at}
            empty_data = empty_data.append(tweet_elem, ignore_index = True)
            
    empty_data.to_csv("pope_with_likes.csv", mode="a")

In [14]:
for i in range(chunks):
        batch = links_to_scrape[i*50:(i+1)*50]
        result = fetch_tw(batch)

In [15]:
pope_likes = pd.read_csv("pope_with_likes.csv")

In [24]:
pope_likes = pope_likes.drop_duplicates(subset = 'tweet_id')

In [25]:
pope_likes.shape

(2878, 6)